In [1]:
import pandas as pd
import numpy as np
import datetime

# Key Assumptions
'curve_year' determines the name of the output file, as well as which pre-generated transportation load curve dataset will be imported.
'pct_heat_elec' is the proportion of heating demand met by heat pumps in that year.
'ASHP_weight' is the proportion of air-source heat pumps (the remainder is assumed to be entirely ground-source heat pumps, with no significant impact from water-source heat pumps)

For the first run, pct_heat_elec was set to 0.35 for 2030 and 0.9 for 2050. ASHP_weight is 0.75 in both cases.

In [30]:
curve_year = 2030
pct_heat_elec = 0.35
ASHP_weight = 0.75

# Base load profiles

In [31]:
base_load = pd.read_csv("../data/load.csv")
base_load.head()

,utc_timestamp,AT,BE,BG,CH,CZ,DE,DK,EE,ES,...,LU,LV,NL,NO,PL,PT,RO,SE,SI,SK
0,01/01/2013 00:00,7273.008880,9208.107529,4488.833468,7061.149722,5597.287520,48551.65719,3324.126087,844.0,24233.29282,...,417,645,11143.84289,14459.27060,11078.05489,5284.386719,5555.529302,15026.92663,1394.275471,2759.402805
1,01/01/2013 01:00,6998.534542,8695.600383,4407.401556,6999.661910,5414.010719,45819.10995,3250.522340,812.0,22954.22955,...,401,616,10886.98953,14282.55865,10955.58648,4925.629264,5549.334418,14889.70064,1332.854905,2746.010081
2,01/01/2013 02:00,6788.500009,8334.247285,4351.053371,6973.980931,5073.513874,43989.75526,3218.772187,788.0,21812.83294,...,390,600,10747.30159,14200.83321,11044.30141,4675.745742,5584.048552,14808.35464,1301.375283,2742.382698
3,01/01/2013 03:00,6601.450469,8161.085044,4282.927052,6672.768370,5686.432477,42984.65536,3225.112615,786.0,20952.52884,...,382,611,10633.91753,14160.19004,11346.50780,4523.547777,5692.312839,14764.83502,1280.570001,2749.075217
4,01/01/2013 04:00,6335.379642,8113.800260,4220.904102,6482.876940,6105.381670,42623.67619,3272.787133,793.0,20593.44090,...,383,614,10415.03684,13971.56183,12278.38921,4423.696561,5961.172064,14843.00911,1289.456382,2752.031094


In [32]:
def update_timestamp(row):
    return pd.to_datetime(row['utc_timestamp']).replace(year = curve_year, tzinfo = None)

In [33]:
base_load['utc_timestamp'] = base_load.apply(lambda r: update_timestamp(r), axis=1)
base_load.set_index('utc_timestamp', inplace=True)
base_load

,AT,BE,BG,CH,CZ,DE,DK,EE,ES,FI,...,LU,LV,NL,NO,PL,PT,RO,SE,SI,SK
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2030-01-01 00:00:00,7273.008880,9208.107529,4488.833468,7061.149722,5597.287520,48551.65719,3324.126087,844.0,24233.29282,10046.221190,...,417,645,11143.84289,14459.27060,11078.05489,5284.386719,5555.529302,15026.92663,1394.275471,2759.402805
2030-01-01 01:00:00,6998.534542,8695.600383,4407.401556,6999.661910,5414.010719,45819.10995,3250.522340,812.0,22954.22955,9905.462216,...,401,616,10886.98953,14282.55865,10955.58648,4925.629264,5549.334418,14889.70064,1332.854905,2746.010081
2030-01-01 02:00:00,6788.500009,8334.247285,4351.053371,6973.980931,5073.513874,43989.75526,3218.772187,788.0,21812.83294,9811.196155,...,390,600,10747.30159,14200.83321,11044.30141,4675.745742,5584.048552,14808.35464,1301.375283,2742.382698
2030-01-01 03:00:00,6601.450469,8161.085044,4282.927052,6672.768370,5686.432477,42984.65536,3225.112615,786.0,20952.52884,9726.668139,...,382,611,10633.91753,14160.19004,11346.50780,4523.547777,5692.312839,14764.83502,1280.570001,2749.075217
2030-01-01 04:00:00,6335.379642,8113.800260,4220.904102,6482.876940,6105.381670,42623.67619,3272.787133,793.0,20593.44090,10178.311070,...,383,614,10415.03684,13971.56183,12278.38921,4423.696561,5961.172064,14843.00911,1289.456382,2752.031094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030-12-31 19:00:00,9386.349859,11024.632880,6151.020241,7683.182412,8329.607068,62324.08489,4389.329937,874.0,32791.91685,10157.153970,...,498,826,15198.95066,15708.10472,17908.95435,7855.883107,6877.162015,16529.90459,2017.074313,3064.645211
2030-12-31 20:00:00,8840.382731,10335.600050,5957.600260,7552.342741,7977.239649,60587.59882,4137.483788,822.0,32491.47081,10301.183330,...,471,785,14520.84862,15671.15604,16938.01477,7644.618577,6379.051350,16027.89693,1946.820075,3016.149170
2030-12-31 21:00:00,8387.627887,10764.096610,5530.564648,7629.098578,7765.971762,57680.45007,3888.735148,798.0,30947.26879,10159.702800,...,482,734,13737.29188,15687.53012,16345.81784,7283.272037,5951.060596,15367.42127,1831.116429,2905.938036


# When2Heat Dataset - Import and Cleaning

In [34]:
w2h = pd.read_csv("../data/when2heat.csv", sep = ";", decimal=",")
w2h.head()

,utc_timestamp,cet_cest_timestamp,AT_COP_ASHP_floor,AT_COP_ASHP_radiator,AT_COP_ASHP_water,AT_COP_GSHP_floor,AT_COP_GSHP_radiator,AT_COP_GSHP_water,AT_COP_WSHP_floor,AT_COP_WSHP_radiator,...,SK_heat_demand_water,SK_heat_demand_water_COM,SK_heat_demand_water_MFH,SK_heat_demand_water_SFH,SK_heat_profile_space_COM,SK_heat_profile_space_MFH,SK_heat_profile_space_SFH,SK_heat_profile_water_COM,SK_heat_profile_water_MFH,SK_heat_profile_water_SFH
0,2007-12-31T22:00:00Z,2007-12-31T23:00:00+0100,2.32,2.85,2.12,2.50,3.57,2.09,3.24,4.53,...,128.0,64.0,45.0,19.0,241.0,177.0,201.0,66.0,68.0,13.0
1,2007-12-31T23:00:00Z,2008-01-01T00:00:00+0100,2.32,2.85,2.12,2.50,3.57,2.09,3.24,4.53,...,128.0,64.0,45.0,19.0,241.0,177.0,201.0,66.0,68.0,13.0
2,2008-01-01T00:00:00Z,2008-01-01T01:00:00+0100,2.31,2.84,2.11,2.47,3.55,2.07,3.23,4.52,...,150.0,80.0,46.0,23.0,245.0,180.0,203.0,83.0,71.0,15.0
3,2008-01-01T01:00:00Z,2008-01-01T02:00:00+0100,2.30,2.82,2.11,2.44,3.51,2.06,3.20,4.51,...,147.0,84.0,45.0,19.0,250.0,182.0,209.0,86.0,68.0,12.0
4,2008-01-01T02:00:00Z,2008-01-01T03:00:00+0100,2.29,2.81,2.10,2.41,3.49,2.05,3.19,4.50,...,163.0,85.0,43.0,35.0,267.0,197.0,217.0,88.0,66.0,23.0


In [35]:
w2h['utc_timestamp'] = pd.to_datetime(w2h['utc_timestamp'], format = '%Y-%m-%d %H:%M:%S').apply(lambda x: x.replace(tzinfo=None))

While COP values run to 2020, heat demand data only extends through the end of 2015. Trim down to an 8-year period.

In [36]:
mask = (w2h['utc_timestamp'] >= '2008-01-01') & (w2h['utc_timestamp'] < '2016-01-01') & ~((w2h['utc_timestamp'].dt.month == 2) &(w2h['utc_timestamp'].dt.day ==29))
w2h_trimmed = w2h.loc[mask]
w2h_trimmed.head()

,utc_timestamp,cet_cest_timestamp,AT_COP_ASHP_floor,AT_COP_ASHP_radiator,AT_COP_ASHP_water,AT_COP_GSHP_floor,AT_COP_GSHP_radiator,AT_COP_GSHP_water,AT_COP_WSHP_floor,AT_COP_WSHP_radiator,...,SK_heat_demand_water,SK_heat_demand_water_COM,SK_heat_demand_water_MFH,SK_heat_demand_water_SFH,SK_heat_profile_space_COM,SK_heat_profile_space_MFH,SK_heat_profile_space_SFH,SK_heat_profile_water_COM,SK_heat_profile_water_MFH,SK_heat_profile_water_SFH
2,2008-01-01 00:00:00,2008-01-01T01:00:00+0100,2.31,2.84,2.11,2.47,3.55,2.07,3.23,4.52,...,150.0,80.0,46.0,23.0,245.0,180.0,203.0,83.0,71.0,15.0
3,2008-01-01 01:00:00,2008-01-01T02:00:00+0100,2.30,2.82,2.11,2.44,3.51,2.06,3.20,4.51,...,147.0,84.0,45.0,19.0,250.0,182.0,209.0,86.0,68.0,12.0
4,2008-01-01 02:00:00,2008-01-01T03:00:00+0100,2.29,2.81,2.10,2.41,3.49,2.05,3.19,4.50,...,163.0,85.0,43.0,35.0,267.0,197.0,217.0,88.0,66.0,23.0
5,2008-01-01 03:00:00,2008-01-01T04:00:00+0100,2.29,2.82,2.10,2.41,3.48,2.04,3.19,4.50,...,268.0,117.0,69.0,81.0,268.0,230.0,241.0,121.0,105.0,53.0
6,2008-01-01 04:00:00,2008-01-01T05:00:00+0100,2.27,2.80,2.09,2.38,3.46,2.03,3.17,4.49,...,540.0,189.0,95.0,256.0,315.0,343.0,289.0,196.0,144.0,167.0


### Section 1: Coefficient of Power

In [37]:
cop = w2h_trimmed.filter(regex='utc_timestamp|_COP_')\
    .melt(id_vars = 'utc_timestamp', value_name = 'cop', var_name = 'variable')
cop.head()

,utc_timestamp,variable,cop
0,2008-01-01 00:00:00,AT_COP_ASHP_floor,2.31
1,2008-01-01 01:00:00,AT_COP_ASHP_floor,2.30
2,2008-01-01 02:00:00,AT_COP_ASHP_floor,2.29
3,2008-01-01 03:00:00,AT_COP_ASHP_floor,2.29
4,2008-01-01 04:00:00,AT_COP_ASHP_floor,2.27


In [38]:
cop[['geo','deleteme','heat_source','heat_sink']] = cop['variable'].str.split(pat='_', expand=True)

cop.loc[cop.heat_sink.isin(['radiator','floor']), 'heat_sink'] = 'space'

cop = cop[cop['heat_source'] != 'WSHP']

cop.head()

,utc_timestamp,variable,cop,geo,deleteme,heat_source,heat_sink
0,2008-01-01 00:00:00,AT_COP_ASHP_floor,2.31,AT,COP,ASHP,space
1,2008-01-01 01:00:00,AT_COP_ASHP_floor,2.30,AT,COP,ASHP,space
2,2008-01-01 02:00:00,AT_COP_ASHP_floor,2.29,AT,COP,ASHP,space
3,2008-01-01 03:00:00,AT_COP_ASHP_floor,2.29,AT,COP,ASHP,space
4,2008-01-01 04:00:00,AT_COP_ASHP_floor,2.27,AT,COP,ASHP,space


In [39]:
cop = cop.groupby(['utc_timestamp', 'geo', 'heat_source', 'heat_sink'])\
    .agg('mean').reset_index()

cop.head()

,utc_timestamp,geo,heat_source,heat_sink,cop
0,2008-01-01,AT,ASHP,space,2.575
1,2008-01-01,AT,ASHP,water,2.110
2,2008-01-01,AT,GSHP,space,3.010
3,2008-01-01,AT,GSHP,water,2.070
4,2008-01-01,BE,ASHP,space,3.250


In [40]:
cop.loc[cop['heat_source']=='ASHP', 'cop'] *= ASHP_weight
cop.loc[cop['heat_source']=='GSHP', 'cop'] *= (1-ASHP_weight)

In [41]:
cop = cop.groupby(['utc_timestamp', 'geo', 'heat_sink'])\
    .aggregate('sum')

cop.head()

cop
utc_timestamp geo heat_sink         
2008-01-01    AT  space      2.68375
                  water      2.10000
              BE  space      3.46000
                  water      2.40000
              BG  space      2.73625

### Section 2: Heat Demand

In [42]:
heat_demand = w2h_trimmed.filter(regex='utc_timestamp|_heat_demand_')\
    .melt(id_vars = 'utc_timestamp', value_name = 'heat_demand_MW', var_name = 'variable')

heat_demand.head()

,utc_timestamp,variable,heat_demand_MW
0,2008-01-01 00:00:00,AT_heat_demand_space,9432.0
1,2008-01-01 01:00:00,AT_heat_demand_space,9636.0
2,2008-01-01 02:00:00,AT_heat_demand_space,10198.0
3,2008-01-01 03:00:00,AT_heat_demand_space,10886.0
4,2008-01-01 04:00:00,AT_heat_demand_space,13477.0


In [43]:
heat_demand[['geo','deleteme1','deleteme2','heat_sink','building_type']] = heat_demand['variable'].str.split(pat='_', expand=True)

heat_demand = heat_demand[(heat_demand['building_type'].isnull()) & (heat_demand['heat_sink'] != "total")]

heat_demand = heat_demand[['utc_timestamp', 'geo', 'heat_sink', 'heat_demand_MW']]

heat_demand.head()

,utc_timestamp,geo,heat_sink,heat_demand_MW
0,2008-01-01 00:00:00,AT,space,9432.0
1,2008-01-01 01:00:00,AT,space,9636.0
2,2008-01-01 02:00:00,AT,space,10198.0
3,2008-01-01 03:00:00,AT,space,10886.0
4,2008-01-01 04:00:00,AT,space,13477.0


Norway and Switzerland are missing heat demand information. Set them equal to Sweden and Austria, respectively, then re-scale according to their relative country populations

In [44]:
heat_demand_fill = heat_demand[heat_demand['geo'].isin(['AT','SE'])]

heat_demand_fill.loc[heat_demand_fill['geo']=='AT', 'geo'] = 'CH'
heat_demand_fill.loc[heat_demand_fill['geo']=='CH', 'heat_demand_MW'] *= (8.64/8.92)
heat_demand_fill.loc[heat_demand_fill['geo']=='SE', 'geo'] = 'NO'
heat_demand_fill.loc[heat_demand_fill['geo']=='NO', 'heat_demand_MW'] *= (5.38/10.35)

In [45]:
heat_demand_filled = pd.concat([heat_demand, heat_demand_fill])

### Section 3: Converting heat demand to electricity demand

In [46]:
synthetic_load_heating = pd.merge(cop, heat_demand_filled, on = ['geo', 'utc_timestamp', 'heat_sink'])
synthetic_load_heating.head()

,geo,utc_timestamp,heat_sink,cop,heat_demand_MW
0,AT,2008-01-01,space,2.68375,9432.0
1,AT,2008-01-01,water,2.10000,402.0
2,BE,2008-01-01,space,3.46000,10995.0
3,BE,2008-01-01,water,2.40000,491.0
4,BG,2008-01-01,space,2.73625,2011.0


This section first converts heat demand to electricity demand. Then, it sums the electricity demand across space and water within a particular hour. Next, we convert all the years in the timestamps to 2020 and average over them. This means the final value is the average of a given hour across all the years in the data (2008-2015, unless otherwise updated).

In [47]:
synthetic_load_heating['elec_demand_MW'] = (synthetic_load_heating['heat_demand_MW'] / synthetic_load_heating['cop']) * pct_heat_elec

synthetic_load_heating = synthetic_load_heating.groupby(['utc_timestamp', 'geo'])\
    .aggregate('sum').reset_index()

In [48]:
synthetic_load_heating.head()

,utc_timestamp,geo,cop,heat_demand_MW,elec_demand_MW
0,2008-01-01,AT,4.78375,9834.000000,1297.069865
1,2008-01-01,BE,5.86000,11486.000000,1183.815149
2,2008-01-01,BG,4.83875,2173.000000,284.199508
3,2008-01-01,CH,4.70500,9525.309417,1287.989356
4,2008-01-01,CZ,5.09125,9449.000000,1147.075390


In [49]:
synthetic_load_heating['utc_timestamp'] = synthetic_load_heating.apply(lambda r: update_timestamp(r), axis=1)

synthetic_load_heating = synthetic_load_heating.groupby(['utc_timestamp', 'geo'])\
    .aggregate('mean').reset_index()

In [50]:
synthetic_load_heating = synthetic_load_heating.pivot(index='utc_timestamp', columns='geo', values='elec_demand_MW').reset_index()
synthetic_load_heating.set_index('utc_timestamp', inplace=True)

In [51]:
synthetic_load_heating

geo,AT,BE,BG,CH,CZ,DE,DK,EE,ES,FI,...,LU,LV,NL,NO,PL,PT,RO,SE,SI,SK
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2030-01-01 00:00:00,1040.077829,1089.657982,276.315398,976.409440,1015.986831,8826.313233,574.858606,138.195269,1137.599596,1042.440600,...,83.099474,188.397406,1458.608518,632.637047,2633.526687,109.884363,1198.211006,1224.929276,171.857946,482.014593
2030-01-01 01:00:00,1063.484953,1120.631058,295.983185,996.630370,1036.688827,9029.353482,585.026305,147.076004,1165.498088,1101.110837,...,85.990535,200.596362,1494.432770,641.578832,2689.366554,118.475328,1272.709412,1245.500857,175.570545,491.713995
2030-01-01 02:00:00,1134.518768,1214.727886,346.832281,1060.773674,1106.176991,9696.335325,630.633851,166.156145,1303.598474,1222.106195,...,92.848557,228.239958,1618.169308,681.362054,2880.598756,121.837645,1425.417139,1324.276856,188.639323,522.443142
2030-01-01 03:00:00,1282.724082,1420.647836,489.373998,1198.049763,1258.132666,11229.212949,741.420132,224.539231,1684.175625,1640.380837,...,106.199507,309.817682,1901.797087,775.428349,3321.254842,137.338768,1911.555108,1505.788004,218.292313,575.365273
2030-01-01 04:00:00,1717.780087,1916.716986,596.221685,1607.680625,1687.888761,15140.947155,1028.368282,270.711684,2500.974459,2036.529296,...,139.846641,371.793755,2595.333154,1056.767009,4548.076030,183.340524,2367.555219,2045.056819,300.929528,753.444812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030-12-31 19:00:00,1695.597248,1885.601006,437.031736,1557.651413,1747.826829,15129.526371,1021.598780,192.959832,2212.148727,1409.628616,...,132.883066,268.104490,2609.626796,1102.696476,4667.226050,248.512456,1806.978974,2078.012144,294.764423,748.244948
2030-12-31 20:00:00,1561.551946,1717.203475,367.850008,1440.030354,1606.772084,13795.401237,921.493452,157.814497,2054.884646,1179.933990,...,122.004645,217.843249,2369.981596,1002.723390,4285.880326,250.706865,1533.473590,1898.217667,268.237132,695.575390
2030-12-31 21:00:00,1315.141228,1379.381597,297.945479,1211.934145,1343.234176,11269.653603,737.232909,130.455131,1604.227625,984.485905,...,99.813895,179.506220,1908.349851,818.241540,3535.418459,231.750919,1270.561127,1572.454964,224.587608,600.697012


### Section 4: Importing transportation demand

In [52]:
synthetic_load_transport = pd.read_csv("../data/synthetic_transport_" + str(curve_year) + ".csv")

In [53]:
synthetic_load_transport['utc_timestamp'] = synthetic_load_transport.apply(lambda r: update_timestamp(r), axis=1)

In [54]:
synthetic_load_transport.set_index('utc_timestamp', inplace = True)
synthetic_load_transport

,AT,BE,BG,CH,CZ,DE,DK,EE,ES,FI,...,LU,LV,NL,NO,PL,PT,RO,SE,SI,SK
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2030-01-01 00:00:00,292.466282,384.130417,41.572891,308.384514,211.277837,2344.042848,211.419874,31.913688,445.695720,112.080462,...,44.903850,8.884863,502.793533,444.952998,307.973084,167.196001,63.951663,550.556308,66.093905,41.650838
2030-01-01 01:00:00,193.786831,254.577159,27.542315,204.341584,139.992205,1553.390311,140.131419,21.146223,295.472207,74.258156,...,29.756892,5.887323,333.242406,294.841581,204.064390,110.855656,42.368411,364.848320,43.794088,27.594298
2030-01-01 02:00:00,123.940011,162.905968,17.609308,130.702324,89.535395,993.880234,89.696443,13.525037,189.224274,47.483860,...,19.037653,3.765752,213.281040,188.599808,130.517722,71.015328,27.088460,233.428963,28.010237,17.643091
2030-01-01 03:00:00,83.697050,110.123244,11.883943,88.279043,60.464488,671.665001,60.666943,9.134241,128.108310,32.053837,...,12.864086,2.543549,144.223958,127.398864,88.144861,48.107157,18.281135,157.743726,18.916572,11.907454
2030-01-01 04:00:00,63.301405,83.423854,8.978733,66.785443,45.731428,508.589273,45.997868,6.909272,97.282922,24.228128,...,9.738870,1.924362,109.314273,96.398363,66.672445,36.565976,13.812065,119.435260,14.308327,8.997359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030-12-31 19:00:00,779.323430,1088.974273,106.311443,830.692737,563.550786,6534.033901,618.460360,85.472501,1376.542415,291.583991,...,124.253094,23.981133,1453.056816,1207.103042,824.055681,532.991758,163.553297,1530.171772,176.803102,106.925297
2030-12-31 20:00:00,718.217109,1003.588401,97.975621,765.558577,519.363079,6021.703905,569.967224,78.770649,1268.608483,268.721051,...,114.510477,22.100786,1339.123432,1112.454758,759.441929,491.200168,150.729173,1410.191847,162.940068,98.541344
2030-12-31 21:00:00,637.317692,890.544983,86.939723,679.326654,460.862426,5343.423855,505.766559,69.897984,1125.713409,238.452520,...,101.612106,19.611371,1188.285609,987.148718,673.898980,435.871763,133.751156,1251.348933,144.586625,87.441723


Section 5: Combining load curves

In [55]:
load_curve = pd.concat([base_load, synthetic_load_heating, synthetic_load_transport])\
    .groupby('utc_timestamp')\
    .agg('sum')

In [56]:
load_curve

,AT,BE,BG,CH,CZ,DE,DK,EE,ES,FI,...,LU,LV,NL,NO,PL,PT,RO,SE,SI,SK
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2030-01-01 00:00:00,8605.552991,10681.895928,4806.721758,8345.943677,6824.552188,59722.013271,4110.404567,1014.108957,25816.588137,11200.742252,...,545.003325,842.282268,13105.244941,15536.860646,14019.554662,5561.467083,6817.691971,16802.412214,1632.227321,3283.068236
2030-01-01 01:00:00,8255.806326,10070.808600,4730.927057,8200.633864,6590.691751,56401.853743,3975.680064,980.222227,24415.199845,11080.831210,...,516.747426,822.483685,12714.664705,15218.979063,13849.017424,5154.960248,6864.412242,16500.049817,1552.219538,3265.318375
2030-01-01 02:00:00,8046.958788,9711.881138,4715.494960,8165.456929,6269.226260,54679.970818,3939.102481,967.681182,23305.655688,11080.786210,...,501.886210,832.005710,12578.751938,15070.795072,14055.417888,4868.598715,7036.554151,16366.060459,1518.024843,3282.468930
2030-01-01 03:00:00,7967.871602,9691.856124,4784.184993,7959.097177,7005.029631,54885.533311,4027.199690,1019.673472,22764.812775,11399.102813,...,501.063593,923.361231,12679.938574,15063.017253,14755.907503,4708.993702,7622.149082,16428.366751,1517.778886,3336.347945
2030-01-01 04:00:00,8116.461134,10113.941100,4826.104520,8157.343008,7839.001859,58273.212618,4347.153283,1070.620956,23191.698281,12239.068494,...,532.585511,987.718117,13119.684268,15124.727201,16893.137685,4643.603061,8342.539348,17007.501188,1604.694238,3514.473265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030-12-31 19:00:00,11861.270537,13999.208159,6694.363420,10071.526562,10640.984683,83987.645162,6029.389077,1152.432332,36380.607992,11858.366577,...,755.136160,1118.085624,19261.634272,18017.904238,23400.236081,8637.387321,8847.694286,20138.088506,2488.641838,3919.815457
2030-12-31 20:00:00,11120.151787,13056.391926,6423.425889,9757.931672,10103.374813,80404.703962,5628.944464,1058.585146,35814.963939,11749.838371,...,707.515123,1024.944035,18229.953648,17786.334188,21983.337026,8386.525610,8063.254113,19336.306444,2377.997275,3810.265903
2030-12-31 21:00:00,10340.086807,13034.023190,5915.449850,9520.359376,9570.068364,74293.527528,5131.734616,998.353116,33677.209824,11382.641225,...,683.426001,933.117591,16833.927340,17492.920378,20555.135280,7950.894719,7355.372879,18191.225167,2200.290662,3594.076771


In [57]:
load_curve.to_csv("../data/synthetic_load_" + str(curve_year) + ".csv")

In [24]:
load1 = pd.read_csv('../data/load.csv', parse_dates=True)
load2 = pd.read_csv('../data/synthetic_load_2030.csv', parse_dates=True)
load3 = pd.read_csv('../data/synthetic_load_2050.csv', parse_dates=True)

In [25]:
def update_timezone(row):
    return pd.to_datetime(row['utc_timestamp'], utc = True)
load1['utc_timestamp'] = load1.apply(lambda r: update_timezone(r), axis=1)

In [26]:
pd.concat([load1,load2,load3]).to_csv('../data/load_20_30_50.csv', index = False)